In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [3]:
import warnings
warnings.filterwarnings("ignore")

# Part 1. Install necessary independences

In [4]:
!pip install wrds
!pip install swig
!pip install PyPortfolioOpt
!pip install pyfolio-reloaded
!pip install -q condacolab
!pip install stockstats
!pip install gym
!pip install gymnasium
!pip install stable_baselines3
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig

  Using cached scipy-1.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached scipy-1.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.4 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
osqp 0.6.7 requires scipy!=1.12.0,>=0.13.2, but you have scipy 1.12.0 which is incompatible.
  Using cached scipy-1.13.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached scipy-1.13.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.12.0
    Uninstalling scipy-1.12.0:
      Successfully uninstalled scipy-1.12.0
ERROR: pip's dependency resolver

In [5]:
import sys
import os
import itertools
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime
from plot import backtest_plot, get_baseline, backtest_stats
%matplotlib inline
from preprocessor.preprocessors import FeatureEngineer, data_split
from env.env_stocktrading import StockTradingEnv
from models.models import DRLAgent,DRLEnsembleAgent
from stable_baselines3.common.logger import configure
from stable_baselines3 import A2C, DDPG, PPO

from pprint import pprint

import itertools

In [6]:
from config.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
    check_and_make_directories
)
from config.config_tickers import DOW_30_TICKER

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

# Part 2. Read data

We first read the .csv file of our processed data into dataframe.

In [7]:
processed_data = pd.read_csv(DATA_SAVE_DIR + 'processed_full.csv')
processed_data = processed_data.set_index(processed_data.columns[0])
processed_data.index.names = ['']

In [ ]:
print(f"Train date: {TRAIN_START_DATE} - {TRAIN_END_DATE}")
print(f"Test date: {TEST_START_DATE} - {TEST_END_DATE}")
print(f"Trade date: {TRADE_START_DATE} - {TRADE_END_DATE}")

In [8]:
train = data_split(processed_data, TRAIN_START_DATE,TRAIN_END_DATE)
print(len(train))

29232
14500


In [9]:
TRAIN_END_DATE

'2022-01-01'

In [10]:
train.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,dx_30,vix,turbulence
0,2018-01-02,AAPL,42.540001,43.075001,42.314999,40.670971,102223600.0,1.0,0.0,0.0,66.666667,100.0,9.77,0.0
0,2018-01-02,AMGN,175.350006,177.820007,174.419998,146.429108,2301100.0,1.0,0.0,0.0,66.666667,100.0,9.77,0.0
0,2018-01-02,AXP,99.730003,99.730003,98.220001,90.226944,2746700.0,1.0,0.0,0.0,66.666667,100.0,9.77,0.0
0,2018-01-02,BA,295.750000,296.989990,295.399994,282.886414,2978900.0,1.0,0.0,0.0,66.666667,100.0,9.77,0.0
0,2018-01-02,CAT,158.300003,159.389999,156.029999,134.870636,5108400.0,1.0,0.0,0.0,66.666667,100.0,9.77,0.0


In [11]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,dx_30,vix,turbulence
1007,2021-12-31,UNH,504.140015,506.869995,502.040009,484.463928,1695400.0,4.0,12.928914,66.917702,105.219094,35.524631,17.219999,12.137917
1007,2021-12-31,V,216.809998,217.979996,215.490005,212.991302,4723300.0,4.0,2.616086,51.608553,86.144479,0.410017,17.219999,12.137917
1007,2021-12-31,VZ,52.200001,52.310001,51.880001,44.412312,15911400.0,4.0,0.200583,48.646226,41.490886,10.681208,17.219999,12.137917
1007,2021-12-31,WBA,51.900002,52.419998,51.830002,45.353912,3406700.0,4.0,0.966713,60.665078,137.655018,25.778533,17.219999,12.137917
1007,2021-12-31,WMT,47.733334,48.346668,47.639999,46.611462,17947800.0,4.0,0.035970,52.959531,77.948767,1.606327,17.219999,12.137917


In [12]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,dx_30,vix,turbulence
0,2022-01-03,AAPL,177.830002,182.880005,177.710007,179.724533,104487900.0,0.0,4.767874,64.844852,102.677577,42.087725,16.6,40.628291
0,2022-01-03,AMGN,223.630005,226.970001,222.029999,210.740341,2742800.0,0.0,4.504905,60.592151,90.703115,20.989652,16.6,40.628291
0,2022-01-03,AXP,164.509995,168.399994,164.399994,162.605331,3236400.0,0.0,-0.638614,50.972514,59.310205,1.754196,16.6,40.628291
0,2022-01-03,BA,204.000000,210.550003,203.339996,207.860001,9060200.0,0.0,-0.982440,49.747896,65.232126,3.639708,16.6,40.628291
0,2022-01-03,CAT,207.330002,208.600006,205.800003,196.558151,2055600.0,0.0,1.291196,53.031031,98.541070,6.201519,16.6,40.628291


In [13]:
trade.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,dx_30,vix,turbulence
499,2023-12-28,UNH,523.469971,527.869995,522.940002,520.630310,2001000.0,3.0,-4.482944,48.544493,-86.554479,9.418188,12.47,7.060585
499,2023-12-28,V,258.540009,260.970001,258.540009,259.915344,3020500.0,3.0,2.767469,61.502711,102.513885,26.012559,12.47,7.060585
499,2023-12-28,VZ,37.180000,37.720001,37.150002,36.280758,14931700.0,3.0,0.248021,57.521271,-7.145725,15.051402,12.47,7.060585
499,2023-12-28,WBA,26.590000,27.020000,26.389999,25.939976,7899900.0,3.0,1.399880,64.322401,118.533676,35.833180,12.47,7.060585
499,2023-12-28,WMT,52.590000,52.776669,52.500000,52.345058,16776000.0,3.0,-0.069524,50.536042,89.737088,0.984013,12.47,7.060585


In [14]:
print(f"Train shape: {train.shape}")
print(f"Trade shape: {trade.shape}")

Train shape: (29232, 14)
Trade shape: (14500, 14)


# Part 3. Construct the environment

Calculate and specify the parameters we need for constructing the environment.

In [15]:
INDICATORS = ['macd',
            'rsi_30',
            'cci_30',
            'dx_30']

In [16]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 175


In [17]:
train

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,dx_30,vix,turbulence
0,2018-01-02,AAPL,42.540001,43.075001,42.314999,40.670971,102223600.0,1.0,0.000000,0.000000,66.666667,100.000000,9.770000,0.000000
0,2018-01-02,AMGN,175.350006,177.820007,174.419998,146.429108,2301100.0,1.0,0.000000,0.000000,66.666667,100.000000,9.770000,0.000000
0,2018-01-02,AXP,99.730003,99.730003,98.220001,90.226944,2746700.0,1.0,0.000000,0.000000,66.666667,100.000000,9.770000,0.000000
0,2018-01-02,BA,295.750000,296.989990,295.399994,282.886414,2978900.0,1.0,0.000000,0.000000,66.666667,100.000000,9.770000,0.000000
0,2018-01-02,CAT,158.300003,159.389999,156.029999,134.870636,5108400.0,1.0,0.000000,0.000000,66.666667,100.000000,9.770000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,2021-12-31,UNH,504.140015,506.869995,502.040009,484.463928,1695400.0,4.0,12.928914,66.917702,105.219094,35.524631,17.219999,12.137917
1007,2021-12-31,V,216.809998,217.979996,215.490005,212.991302,4723300.0,4.0,2.616086,51.608553,86.144479,0.410017,17.219999,12.137917
1007,2021-12-31,VZ,52.200001,52.310001,51.880001,44.412312,15911400.0,4.0,0.200583,48.646226,41.490886,10.681208,17.219999,12.137917
1007,2021-12-31,WBA,51.900002,52.419998,51.830002,45.353912,3406700.0,4.0,0.966713,60.665078,137.655018,25.778533,17.219999,12.137917


In [18]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [19]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Part 4. Train DRL Agents

In [20]:
agent = DRLAgent(env = env_train)

# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_ensemble = True

### Agent 1: A2C

In [21]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + 'a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
Logging to results/a2c


In [22]:
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=20000) if if_using_a2c else None

---------------------------------------
| time/                 |             |
|    fps                | 65          |
|    iterations         | 100         |
|    time_elapsed       | 7           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -41.2       |
|    explained_variance | 0.698       |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | 23.1        |
|    reward             | 0.024734754 |
|    std                | 1           |
|    value_loss         | 0.341       |
---------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 70        |
|    iterations         | 200       |
|    time_elapsed       | 14        |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -41.2     |
|    explained_variance | 0.243     |
|    learning_rate

In [23]:
trained_a2c.save(TRAINED_MODEL_DIR + "agent_a2c") if if_using_a2c else None

### Agent 2: DDPG

In [24]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + 'ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device
Logging to results/ddpg


In [25]:
trained_ddpg = agent.train_model(model=model_ddpg,
                            tb_log_name='ddpg',
                            total_timesteps=20000) if if_using_ddpg else None

Episode: 22
Episode: 23
Episode: 24
Episode: 25
------------------------------------
| time/              |             |
|    episodes        | 4           |
|    fps             | 82          |
|    time_elapsed    | 48          |
|    total_timesteps | 4032        |
| train/             |             |
|    actor_loss      | 53.4        |
|    critic_loss     | 178         |
|    learning_rate   | 0.001       |
|    n_updates       | 3931        |
|    reward          | -0.04534058 |
------------------------------------
Episode: 26
Episode: 27
Episode: 28
Episode: 29
------------------------------------
| time/              |             |
|    episodes        | 8           |
|    fps             | 84          |
|    time_elapsed    | 95          |
|    total_timesteps | 8064        |
| train/             |             |
|    actor_loss      | 40.8        |
|    critic_loss     | 0.692       |
|    learning_rate   | 0.001       |
|    n_updates       | 7963        |
|    reward     

In [26]:
trained_ddpg.save(TRAINED_MODEL_DIR + "agent_ddpg") if if_using_ddpg else None

### Agent 3: PPO

In [27]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + 'ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cuda device
Logging to results/ppo


In [28]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=20000) if if_using_ppo else None

Episode: 42
Episode: 43
-----------------------------------
| time/              |            |
|    fps             | 177        |
|    iterations      | 1          |
|    time_elapsed    | 11         |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | 0.05710453 |
-----------------------------------
Episode: 44
Episode: 45
-----------------------------------------
| time/                   |             |
|    fps                  | 169         |
|    iterations           | 2           |
|    time_elapsed         | 24          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.017482497 |
|    clip_fraction        | 0.213       |
|    clip_range           | 0.2         |
|    entropy_loss         | -41.2       |
|    explained_variance   | -0.0424     |
|    learning_rate        | 0.00025     |
|    loss                 | 3.26        |
|    n_updates            | 10          

In [29]:
trained_ppo.save(TRAINED_MODEL_DIR + "agent_ppo") if if_using_ppo else None

## Ensemble Strategy

In [30]:
ensemble_data = processed_data

In [31]:
rebalance_window = 63
validation_window = 63

env_ensemble_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "print_verbosity":5
}

ensemble_agent = DRLEnsembleAgent(df=ensemble_data,
                 train_period=(TRAIN_START_DATE,TRAIN_END_DATE),
                 val_test_period=(TEST_START_DATE,TEST_END_DATE),
                 rebalance_window=rebalance_window,
                 validation_window=validation_window,
                 **env_ensemble_kwargs) if if_using_ensemble else None

In [32]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.005,
                    'learning_rate': 0.0007
                    }

PPO_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.00025,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      #"action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 10_000,
                      "learning_rate": 0.0005,
                      "batch_size": 64
                    }

timesteps_dict = {'a2c' : 20000,
                 'ppo' : 20000,
                 'ddpg' : 20000,
                 }

In [33]:
df_summary = ensemble_agent.run_ensemble_strategy(A2C_model_kwargs,
                                                 PPO_model_kwargs,
                                                 DDPG_model_kwargs,
                                                 timesteps_dict) if if_using_ensemble else None

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
|    explained_variance | -0.0144   |
|    learning_rate      | 0.0007    |
|    n_updates          | 999       |
|    policy_loss        | -254      |
|    reward             | 2.6416326 |
|    std                | 0.994     |
|    value_loss         | 42.6      |
-------------------------------------
Episode: 5
day: 1070, episode: 5
begin_total_asset: 1000000.00
end_total_asset: 1568851.39
total_reward: 568851.39
total_cost: 61100.22
total_trades: 21003
Sharpe: 0.618
--------------------------------------
| time/                 |            |
|    fps                | 150        |
|    iterations         | 1100       |
|    time_elapsed       | 36         |
|    total_timesteps    | 5500       |
| train/                |            |
|    entropy_loss       | -41        |
|    explained_variance | 0.0575     |
|    learning_rate      | 0.0007     |
|    n_updates          | 1099       |
|    policy_loss        | -27.9      |
|

In [34]:
df_summary if if_using_ensemble else None

,Iter,Val Start,Val End,Model Used,A2C Sharpe,PPO Sharpe,DDPG Sharpe
0,126,2022-01-03,2022-04-04,A2C,-0.024746,-0.108454,-0.068447
1,189,2022-04-04,2022-07-06,A2C,-0.160437,-0.259066,-0.25292
2,252,2022-07-06,2022-10-04,A2C,-0.080968,-0.185529,-0.094428
3,315,2022-10-04,2023-01-04,A2C,0.380992,0.18812,0.358399
4,378,2023-01-04,2023-04-05,DDPG,-0.050632,-0.043434,0.107677
5,441,2023-04-05,2023-07-07,A2C,0.303229,-0.060818,-0.082253
